In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn

from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score

from sklearn.base import BaseEstimator

from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

### 머신러닝 모델 평가
+ 일반적으로 머신러닝은
    - 데이터가공/변환(전처리)
    - 모델 학습/예측
    - 평가의 과정을 거침
+ 앞의 타이타닉분석에서 모델의 평가는 정확도만 사용함
+ 한편, 머신러닝의 예측성능의 평가방법은
    - 회귀 : R^2, MSE평균제곱오차
    - 분류 : 오차행렬, ROC, AUC, F1스코어, 크로스엔트로피, 최대우도

### 정확도의 함정
+ 탐색적 분석 시행시 성별 기준 생존비율은 여성이 더 높음
+ 따라서, 굳이 ML알고리즘을 적용하지 않아도 성별이 여성일 경우 생존, 남성일 경우 사망이라고 예측해도 크게 무리가 없음
+ 즉, 단순히 성별 조건 하나만 적용해도 별거 아닌 알고리즘으로도 높은 정확도가 나타나는 상황 발생

In [2]:
# BaseEstimator 클래스를 상속받아 가짜 분류기 생성

class DummyClassifier(BaseEstimator):
    def fit(self, X, y=None):
        pass
    """
    성별이 1(남자)이면 사망(0)
    성별이 0(여자)이면 생존(1)이라고
    예측하는 predict메서드 정의 """
    def predict(self, X):
        # 입력데이터 크기만큼 0으로 채워진 1차원 행렬 생성
        pred = np.zeros((X.shape[0], 1))
        # 성별이 여성이라면 무족건 생존이라 예측
        for i in range(X.shape[0]):
            if X['sex'].iloc[i] != 1:
                pred[i] = 1
        return pred
    

In [3]:
titanic = pd.read_csv('csv/titanic2.csv')
titanic['sex'] = titanic['sex'].apply(lambda x: 0 if x == 'female' else 1)

In [4]:
data = titanic.iloc[:, [0, 3,4, 5, 6, 8]]
target = titanic.survived
Xtrain, Xtest, ytrain, ytest = train_test_split(data, target, train_size=0.7, random_state=2111041205)

In [5]:
myclf = DummyClassifier()
myclf.fit(Xtrain, ytrain)
pred = myclf.predict(Xtest)
accuracy_score(pred,ytest)

0.798469387755102

+ 따라서, 불균형한 레이블 데이터셋트의 성능수치로 정확도를 평가지표로 사용해서는 안됨
    - 이러한 한계를 극복하기 위해 오차행렬을 사용
    - 특히 분류모델에서는 정확도보다 정밀도/재현율을 더 선호

In [6]:
confusion_matrix(pred, ytest)

array([[202,  36],
       [ 43, 111]])

In [7]:
print(classification_report(pred, ytest))

              precision    recall  f1-score   support

         0.0       0.82      0.85      0.84       238
         1.0       0.76      0.72      0.74       154

    accuracy                           0.80       392
   macro avg       0.79      0.78      0.79       392
weighted avg       0.80      0.80      0.80       392



![nn](jpg/CM.jpg)

### iris 데이터셋을 이용해서 로지스틱 회귀모델로 분석한 후 오차행렬과 정밀도/재현율 등을 확인해보세요.

In [8]:
from sklearn.datasets import load_iris
iris = load_iris()
Xtrain, Xtest, ytrain, ytest = train_test_split(iris.data, iris.target, train_size=0.7, stratify=iris.target, random_state=2111041255)

In [9]:
lrclf = LogisticRegression()
lrclf.fit(Xtrain, ytrain)
pred = lrclf.predict(Xtest)
accuracy_score(pred,ytest)

0.9777777777777777

In [10]:
confusion_matrix(pred, ytest)

array([[15,  0,  0],
       [ 0, 14,  0],
       [ 0,  1, 15]])

In [11]:
print(classification_report(pred, ytest))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        15
           1       0.93      1.00      0.97        14
           2       1.00      0.94      0.97        16

    accuracy                           0.98        45
   macro avg       0.98      0.98      0.98        45
weighted avg       0.98      0.98      0.98        45



### 정확도
+ 맞는 것을 맞다고, 틀린것을 틀리다고 올바르게 예측한 것
+ $ accuacry = \frac{TP+TN}{TP+TN+FP+FN} $
+ 오차행렬에서 대각선 방향

In [12]:
# 타이타닉 분석결과의 오차행렬 정확도 계산
(202+111) / (202+36+43+111)

0.798469387755102

### 정밀도
+ 모델의 예측값이 얼마나 올바른지 알아보는 지표
+ 양성으로 예측한 것 중에서 실제로 양성인 것은 얼마나 존재하나?
+ $ precision = \frac {TP} {TP+FP} $

In [13]:
202 / (202+43)

0.8244897959183674

### 6마리의 동물형상 중 개p/고양이n를 맞추는 게임을 진행
### 정답 = [개     개     개 고양이 개     고양이]
### 예측 = [개     고양이 개 고양이 개     개    ]

+ 정확도는?  4 / 6 = 0.67 (개를 개라고, 고양이를 고양이라고 예측)
+ 정밀도는?  3 / 4 = 0.75 (개라고 예측한 것 중에서 실제 개인 비율)
+ 재현율는?  3 / 4 = 0.75 (실제 개중에서 내가 개라고 예측한 것)

In [14]:
"""
오차행렬은?
            p  n  (predict)
           
(actual) p  3  1

         n  1  1"""

'\n오차행렬은?\n            p  n  (predict)\n           \n(actual) p  3  1\n\n         n  1  1'

### 178개의 와인샘플을 13개의 여러 화학적 성분에 근거해
+ 3가지 등급으로 분류한 wine 데이터셋을 이용해서 logistic regression으로 분석하고 오차행렬과 정밀도/재현율, F1스코어등을 확인해보세요

In [15]:
from sklearn.datasets import load_wine
wine = load_wine()
print(wine.DESCR)

.. _wine_dataset:

Wine recognition dataset
------------------------

**Data Set Characteristics:**

    :Number of Instances: 178 (50 in each of three classes)
    :Number of Attributes: 13 numeric, predictive attributes and the class
    :Attribute Information:
 		- Alcohol
 		- Malic acid
 		- Ash
		- Alcalinity of ash  
 		- Magnesium
		- Total phenols
 		- Flavanoids
 		- Nonflavanoid phenols
 		- Proanthocyanins
		- Color intensity
 		- Hue
 		- OD280/OD315 of diluted wines
 		- Proline

    - class:
            - class_0
            - class_1
            - class_2
		
    :Summary Statistics:
    
    ============================= ==== ===== ======= =====
                                   Min   Max   Mean     SD
    ============================= ==== ===== ======= =====
    Alcohol:                      11.0  14.8    13.0   0.8
    Malic Acid:                   0.74  5.80    2.34  1.12
    Ash:                          1.36  3.23    2.36  0.27
    Alcalinity of Ash:            1

In [16]:
wine.data[:5]

wine.feature_names  # 컬럼명
wine.target_names   # 종속변수값
pd.Series(wine.target).value_counts()   # 종속변수값 분포

1    71
0    59
2    48
dtype: int64

In [17]:
# wine 데이터셋을 dataframe으로 생성
wines = pd.DataFrame(wine.data)
wines.columns = wine.feature_names
wines['class'] = wine.target
wines.head()

,alcohol,malic_acid,ash,alcalinity_of_ash,magnesium,total_phenols,flavanoids,nonflavanoid_phenols,proanthocyanins,color_intensity,hue,od280/od315_of_diluted_wines,proline,class
0,14.23,1.71,2.43,15.6,127.0,2.80,3.06,0.28,2.29,5.64,1.04,3.92,1065.0,0
1,13.20,1.78,2.14,11.2,100.0,2.65,2.76,0.26,1.28,4.38,1.05,3.40,1050.0,0
2,13.16,2.36,2.67,18.6,101.0,2.80,3.24,0.30,2.81,5.68,1.03,3.17,1185.0,0
3,14.37,1.95,2.50,16.8,113.0,3.85,3.49,0.24,2.18,7.80,0.86,3.45,1480.0,0
4,13.24,2.59,2.87,21.0,118.0,2.80,2.69,0.39,1.82,4.32,1.04,2.93,735.0,0


In [18]:
wines.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 178 entries, 0 to 177
Data columns (total 14 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   alcohol                       178 non-null    float64
 1   malic_acid                    178 non-null    float64
 2   ash                           178 non-null    float64
 3   alcalinity_of_ash             178 non-null    float64
 4   magnesium                     178 non-null    float64
 5   total_phenols                 178 non-null    float64
 6   flavanoids                    178 non-null    float64
 7   nonflavanoid_phenols          178 non-null    float64
 8   proanthocyanins               178 non-null    float64
 9   color_intensity               178 non-null    float64
 10  hue                           178 non-null    float64
 11  od280/od315_of_diluted_wines  178 non-null    float64
 12  proline                       178 non-null    float64
 13  class

In [19]:
# 데이터들을 train/test로 나눔

Xtrain, Xtest, ytrain, ytest = train_test_split(wine.data, wine.target,train_size=0.7, stratify=wine.target,random_state=2111041645)

In [20]:
# max_iter : 적절한 해를 찾기 위한 탐색횟수 지정
lrclf = LogisticRegression(max_iter=2500)
lrclf.fit(Xtrain, ytrain)
pred = lrclf.predict(Xtest)
accuracy_score(pred, ytest) # 0.94

0.9444444444444444

In [21]:
confusion_matrix(pred, ytest)

array([[17,  0,  0],
       [ 1, 20,  1],
       [ 0,  1, 14]])

In [22]:
print(classification_report(pred, ytest))

              precision    recall  f1-score   support

           0       0.94      1.00      0.97        17
           1       0.95      0.91      0.93        22
           2       0.93      0.93      0.93        15

    accuracy                           0.94        54
   macro avg       0.94      0.95      0.94        54
weighted avg       0.94      0.94      0.94        54



In [23]:
# 랜덤 포레스트를 이용한 분석
from sklearn.ensemble import RandomForestClassifier

In [24]:
rfclf = RandomForestClassifier()
rfclf.fit(Xtrain, ytrain)
pred = rfclf.predict(Xtest)
accuracy_score(pred, ytest) # 0.94

0.9629629629629629

In [25]:
confusion_matrix(pred, ytest)

array([[18,  0,  0],
       [ 0, 19,  0],
       [ 0,  2, 15]])

In [26]:
print(classification_report(pred, ytest))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        18
           1       0.90      1.00      0.95        19
           2       1.00      0.88      0.94        17

    accuracy                           0.96        54
   macro avg       0.97      0.96      0.96        54
weighted avg       0.97      0.96      0.96        54



### 적절한 변수선택을 통한 분석 방법
* SequentialFeatureSelector 를 이용해서 적절한 변수는 몇개이고 그때의 정확도는 얼마인지 알아볼수 있음

In [27]:
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.neighbors import KNeighborsClassifier

In [28]:
scores = []
#lrclf = LogisticRegression(max_iter=2500)
knn = KNeighborsClassifier()
for n in range(1, 13):
   sfs = SequentialFeatureSelector(knn,
       n_features_to_select=n, n_jobs=-1)
   sfs.fit(Xtrain, ytrain)
   mask = sfs.support_
   knn.fit(Xtrain[:, mask], ytrain)    
   scores.append(knn.score(Xtrain[:, mask], ytrain))

In [29]:
# 종속변수가 6~7일때 정확도가 좋음
sfs = SequentialFeatureSelector(knn, n_features_to_select=6, n_jobs=-1)
sfs.fit(Xtrain, ytrain)
mask = sfs.support_
knn.fit(Xtrain[:, mask], ytrain)
knn.score(Xtrain[:, mask], ytrain)  # 0.959

0.9596774193548387

In [30]:
# 최적의 정확도 출력시 사용한 컬럼들
wines.columns[1:][mask] 

Index(['alcalinity_of_ash', 'flavanoids', 'nonflavanoid_phenols',
       'proanthocyanins', 'color_intensity', 'hue'],
      dtype='object')